# Auction Simluation Example

This is an example of how the [Economic Simluation Library (ESL)](https://github.com/EconomicSL) can be integrated into your standard python environment. 

Using only python, we will run an example auction simulation, and then use [pandas](http://pandas.pydata.org/) to analyse the results.

Please note: this will require you to have Scala and Java installed on your computer, more information can be found in the [GitHub repo](https://github.com/EconomicSL/auctions-simulation-example).

## Setup

In [1]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import collections
import json

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
cf.go_offline()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


## Run the simulation 

The simulation is a scala programme, and the results are written to a local file called `output.json`.

In [2]:
!sbt "run-main ContinuousDoubleAuction"

[warn] Executing in batch mode.
[warn]   For better performance, hit [ENTER] to switch to interactive mode, or
[warn]   consider launching sbt without any commands, or explicitly passing 'shell'
[info] Loading project definition from /Users/ibillett/Projects/auctions-simulation-example/project
[info] Set current project to auctions-simulation-example (in build file:/Users/ibillett/Projects/auctions-simulation-example/)
[info] Running ContinuousDoubleAuction 
[success] Total time: 2 s, completed 04-Jun-2017 21:45:25


## Load the simulation output

The simulation creates a nested JSON file, containing information about each individual trade that was settled during the and filled.

We can load this into pandas and analyse the results we gave generated.

In [3]:
with open('output.json') as simulationOutput:    
    data = json.load(simulationOutput)
    
df = pd.io.json.json_normalize(data)

With the data loaded into pandas, we can take a look at some of the simulation output.

In [4]:
df.head()

,askOrder.issuer,askOrder.limit.value,askOrder.quantity.value,askOrder.tradable.tick,bidOrder.issuer,bidOrder.limit.value,bidOrder.quantity.value,bidOrder.tradable.tick,price.value
0,7f9de1a9-7639-4333-a5c8-1de6d2b9c5a0,102948884,1,1,949c4b48-636c-4c6f-a108-0f9fe4c4c2f7,676262207,1,1,674661567
1,da3ed11a-d0b2-4567-a6f3-9c36d66f68ac,1227545275,1,1,9bc6afae-5af6-45b2-92cd-3f2e8a7ba00c,1681268743,1,1,1251951003
2,4d34ed5d-48be-4e2c-bee9-02a7860e37a3,326327863,1,1,a1567139-d5c3-4317-9002-0ae546c5ab51,676774046,1,1,676518127
3,99c3265e-373e-4297-b560-002521628492,1226299823,1,1,cb62ef44-e03e-4802-b9a6-0f510557a5aa,1610411243,1,1,1226922549
4,109fe86b-a39f-4309-b44b-a1f0b4f99ddc,369835212,1,1,69732536-e006-4aab-a971-8a26f3885259,701566201,1,1,689170124


In [5]:
df.describe()

,askOrder.limit.value,askOrder.quantity.value,askOrder.tradable.tick,bidOrder.limit.value,bidOrder.quantity.value,bidOrder.tradable.tick,price.value
count,3.918000e+03,3918,3918,3.918000e+03,3918,3918,3.918000e+03
mean,8.429955e+08,1,1,1.314118e+09,1,1,1.072899e+09
std,4.866433e+08,0,0,4.859733e+08,0,0,3.386128e+08
min,1.712000e+04,1,1,2.795470e+08,1,1,1.545687e+08
25%,4.139539e+08,1,1,8.930750e+08,1,1,7.856974e+08
50%,8.473279e+08,1,1,1.324776e+09,1,1,1.075602e+09
75%,1.262891e+09,1,1,1.729463e+09,1,1,1.353433e+09
max,1.733015e+09,1,1,2.146966e+09,1,1,1.754388e+09


We can easily calculate the spread of each trade that was settled:

In [6]:
df['spread'] = df['bidOrder.limit.value'] - df['askOrder.limit.value']

# Times series plot of price data

Ideally, we should plot the `askOrder.limit.value`, `bidOrder.limit.value`, and the `price.value` on one plot.  Should be the case that the `price.value` is a weighted average of the other two prices.

In [8]:
df[['askOrder.limit.value', 'bidOrder.limit.value', 'price.value']].iplot();

In [9]:
df.spread.iplot(logy=True);

In [10]:
returns = df['price.value'].pct_change()
returns.abs().iplot(logy=True);

In [11]:
# TODO: interactive timeseries plot of the data using plotly (or similar)

# Histogram of the distribution of `askOrder.limit.value`

Useful to see how we can create historgram of values across agents.  Agents are representing by `askOrder.issuer` identifiers.

In [12]:
# limit price for ask orders was sampled from U[1, 2147483647]
df['askOrder.limit.value'].iplot(kind='hist')

In [13]:
# TODO: interactive histogram of the data using plotly (or similar)